In [96]:
import os
import pickle
from tqdm import tqdm, trange
import os
import numpy as np
from collections import defaultdict

In [97]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("darkgrid")

In [105]:
binding = True
root_path = '/home/alexander/ray_results/cross_play_20K_episodes_binding/2021_03_23/12_37_37'

# binding=False
# root_path = 'home/alexander/ray_results/cross_play_20K_episodes_cheap/2021_03_24/05_30_06'

results_filename = os.path.join('results', 'results_' + '_'.join(root_path.split('/')[-3:]) + '.pickle')
results = pickle.load(open(results_filename, 'rb'))

In [106]:
img_dir = results_filename[:-7]
os.makedirs(img_dir, exist_ok=True)

In [107]:
res_dict = defaultdict(list)
for res in results:
    res_dict[(res[0], res[1])].append(res[3])

In [ ]:
for eval_regime_name, eval_regime_list in [
    ('agents of the same type', 
        (('custom_vs_custom_1', 'custom_vs_custom_1'),  # normal eval
        ('custom_vs_custom_1', 'custom_vs_custom_2'),)   # simplest (1)
    ),
    ('both have incorrect notions',
        (
           ('custom_vs_custom_1', 'default_vs_default'),  # 2.1
           ('default_vs_default', 'custom_vs_custom_1'),   # 2.2
           ('custom_vs_custom_1', 'custom_vs_custom_2'),  # as baseline
        ),
    ),
    ('second has incorrect notions',
        (
           ('default_vs_custom', 'custom_vs_custom_1'),   # 3.1
           ('custom_vs_custom_1', 'default_vs_custom'),  # 4.1
           ('custom_vs_custom_1', 'custom_vs_custom_2'),  # as baseline
        ),
    ),
    ('first has incorrect notions',
        (
           ('default_vs_default', 'default_vs_custom'),   # 3.2
           ('default_vs_custom', 'default_vs_default'),  # 4.2
           ('custom_vs_custom_1', 'custom_vs_custom_2'),  # as baseline
        ),
    ),
]:
    for xlabel, xarray, yarray in [
        ('Utilities skew', np.arange(-1, 1.1, 0.4), np.arange(1, 7)),
        ('Fairness coeff', np.arange(0.0, 0.7, 0.1), np.arange(7, 14)),
        ('Prosociality level', np.arange(0, 1.1, 0.2), np.arange(14, 20)),
        ('Hidden layer size', [5, 10, 20, 40, 80, 160], np.arange(20, 26))
    ]:
        plt.figure(figsize=(10, 7))
        for agent_0_match_regime, agent_1_match_regime in eval_regime_list:
#             welfare_vals = []
#             welfare_stds = []
            welfare_positive_vals = []
            welfare_positive_stds = []
            for y in yarray:
                vals = np.array([res['sum_share_of_max'] for res in res_dict[((agent_0_match_regime, agent_1_match_regime), y)]])
#                 welfare_vals.append(vals.mean())
#                 welfare_stds.append(vals.mean(axis=1).std())
                welfare_positive = (vals > 0)
                welfare_positive_vals.append(welfare_positive.mean())
                welfare_positive_stds.append(welfare_positive.mean(axis=1).std())
#             plt.scatter(xarray, welfare_vals,
#                         label=f'{agent_0_match_regime}, {agent_1_match_regime}')
#             plt.errorbar(xarray, welfare_vals, yerr=welfare_stds)
            plt.scatter(xarray, welfare_positive_vals,
                        label=f'{agent_0_match_regime}, {agent_1_match_regime}')
            plt.errorbar(xarray, welfare_positive_vals, yerr=welfare_positive_stds)
            plt.ylim(0.3, 1)
#             plt.ylabel('Welfare')
            plt.ylabel('% of positive-welfare games')
            plt.xlabel(xlabel)
            suptitle = f'% of positive-welfare games vs {xlabel}, {"Binding comms" if binding else "cheap comms"}, {eval_regime_name}'
            plt.suptitle(suptitle)
            plt.legend()
            plt.savefig(os.path.join(img_dir, suptitle + '.png'), bbox_inches='tight')
os.system(f'tar -czvf {img_dir}.tar {img_dir}')